<a href="https://colab.research.google.com/github/rbondili/Ecommerce_E2E_MLProject/blob/master/01_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align=center><font size=5>Customer segmentation
  
We analyze the content of the [E-commerce database](https://www.kaggle.com/carrie1/ecommerce-data) that contains purchases made by about 4000 customers over a period of one year (from 2010/12/01 to 2011/12/09).
  
We then try to segment the customers into several categories, and try to classify new customers into those categories.

# 1) Data preprocessing

## 1.1) Download data using kaggle API


In [ ]:
# user kaggle name and kaggle key
import os
os.environ['KAGGLE_KEY'] = 'Your Key'
os.environ['KAGGLE_USERNAME'] = 'Your User ID'
#!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd /content/drive/My\ Drive/Colab\ Notebooks/Customer\ Segmentation/ecommerce-data

/content/drive/My Drive/Colab Notebooks/Customer Segmentation/ecommerce-data


In [ ]:
pwd

'/content/drive/My Drive/Colab Notebooks/Customer Segmentation/ecommerce-data'

In [ ]:
!kaggle datasets list -s E-Commerce

ref                                                    title                                              size  lastUpdated          downloadCount  
-----------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
carrie1/ecommerce-data                                 E-Commerce Data                                     7MB  2017-08-17 02:44:30          35273  
nicapotato/womens-ecommerce-clothing-reviews           Women's E-Commerce Clothing Reviews                 3MB  2018-02-03 19:59:19          22967  
olistbr/brazilian-ecommerce                            Brazilian E-Commerce Public Dataset by Olist       43MB  2018-11-29 12:22:57          33722  
benroshan/ecommerce-data                               E-Commerce Data                                    18KB  2020-04-22 11:47:38            122  
roshansharma/online-shoppers-intention                 Online Shopper's Intention                        2

In [ ]:
!kaggle datasets download -d carrie1/ecommerce-data

ecommerce-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/Colab Notebooks/Customer Segmentation/ecommerce-data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/My Drive/Colab Notebooks/Customer Segmentation/ecommerce-data')

In [ ]:
ls /content/drive/My\ Drive/Colab\ Notebooks/Customer\ Segmentation/ecommerce-data

data.csv  data_dropna.csv  df_dropna.csv


In [ ]:
!head data.csv

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850,United Kingdom
536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850,United Kingdom
536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850,United Kingdom
536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850,United Kingdom


##1.3)  Read data + timestamp


In [ ]:
import pandas as pd
address = '/content/drive/My Drive/Colab Notebooks/Customer Segmentation/ecommerce-data/data.csv'

df = pd.read_csv(address, encoding='ISO-8859-1')

In [ ]:
df.shape

(541909, 8)

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [ ]:
df.InvoiceDate = pd.to_datetime(df.InvoiceDate)

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


##1.4) Checking for Missing values


In [ ]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [ ]:
df.isnull().mean() * 100

InvoiceNo       0.000000
StockCode       0.000000
Description     0.268311
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     24.926694
Country         0.000000
dtype: float64

In [ ]:
df_dropna = df.dropna(subset = ['CustomerID'])
df_dropna.isnull().mean() * 100

InvoiceNo      0.0
StockCode      0.0
Description    0.0
Quantity       0.0
InvoiceDate    0.0
UnitPrice      0.0
CustomerID     0.0
Country        0.0
dtype: float64

## 1.5) Checking and removing Duplicates


In [ ]:
df_dropna_duplicate = df_dropna[df_dropna.duplicated()]
df_dropna_duplicate.describe()

,Quantity,UnitPrice,CustomerID
count,5225.000000,5225.000000,5225.000000
mean,2.686507,2.415715,15789.580478
std,28.146550,2.254946,1604.631774
min,-1296.000000,0.080000,12359.000000
25%,1.000000,1.060000,14555.000000
50%,1.000000,1.650000,15919.000000
75%,2.000000,2.950000,17231.000000
max,1440.000000,19.950000,18283.000000


In [ ]:
df_dropna_duplicate.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom
555,536412,22327,ROUND SNACK BOXES SET OF 4 SKULLS,1,2010-12-01 11:49:00,2.95,17920.0,United Kingdom


In [ ]:
# check - should be zero if you done everything right
print("number of duplicates is ", sum(df_dropna.duplicated()))

number of duplicates is  5225


In [ ]:
df_dropna.shape

(406829, 8)

In [ ]:
df_dropna.drop_duplicates(inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_dropna.describe()

,Quantity,UnitPrice,CustomerID
count,401604.000000,401604.000000,401604.000000
mean,12.183273,3.474064,15281.160818
std,250.283037,69.764035,1714.006089
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13939.000000
50%,5.000000,1.950000,15145.000000
75%,12.000000,3.750000,16784.000000
max,80995.000000,38970.000000,18287.000000


## 1.6) Save data to data_dropna.csv file


In [ ]:
relative_path ='/content/data/'#put you path on google drive here where you want to save
# data dataframe => relative_path+'/data.csv'
# data_dropna dataframe => relative_path+'/data_dropna.csv'
#save data and data_dropna

df_dropna.to_csv('/content/drive/My Drive/Colab Notebooks/Customer Segmentation/ecommerce-data/data_dropna.csv')